In [1]:
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 18})
import numpy as np
import pandas as pd
from keras.utils.np_utils import to_categorical
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, load_model
from keras.datasets import mnist
import random
import sys
sys.path.append('../')
from Core_Functions import neural_networks, generate_image_set, multi_hot_encoding

Using TensorFlow backend.


In [2]:
# Load data and split into training and test
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [3]:
# Split training set further into training and validation sets
sample_size = 40000
X_train, X_val, y_train, y_val = train_images[:sample_size], train_images[sample_size:sample_size*2],  train_labels[:sample_size], train_labels[sample_size:sample_size*2]

In [4]:
combo_train_X_A, combo_train_X_B , combo_train_y = generate_image_set.get_left_min_image_combinations_separate(40000, X_train, y_train)
combo_val_X_A, combo_val_X_B, combo_val_y = generate_image_set.get_right_min_image_combinations_separate(20000, X_val, y_val)

In [5]:
# Reshape the data so they can be fed into the network, one-hot-encode the labels

train_images_A = combo_train_X_A.reshape((combo_train_X_A.shape[0], 28, 28, 1))
train_images_A = train_images_A/ 255

train_images_B = combo_train_X_B.reshape((combo_train_X_B.shape[0], 28, 28, 1))
train_images_B = train_images_B/ 255

val_images_A = combo_val_X_A.reshape((combo_val_X_A.shape[0], 28, 28, 1))
val_images_A = val_images_A/255

val_images_B = combo_val_X_B.reshape((combo_val_X_B.shape[0], 28, 28, 1))
val_images_B = val_images_B/255

train_labels = to_categorical(combo_train_y)
val_labels = to_categorical(combo_val_y)

In [6]:
model = neural_networks.build_CNN((28, 56, 1))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 54, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 27, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 25, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 12, 64)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 10, 64)         36928     
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                1

In [7]:
reconstructed_classification_model = keras.models.load_model("image_classifier_CNN")
reconstructed_classification_model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_7 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 64)               

In [8]:
image_A_predictions = reconstructed_classification_model.predict(train_images_A)
image_B_predictions = reconstructed_classification_model.predict(train_images_B)

In [10]:
multi_hot_label_list = []
final_label_list = []
for index in range(len(train_images_A)):
    
    # Retrieve intermediate labels
    label_A = np.argmax(image_A_predictions[index])
    label_B = np.argmax(image_B_predictions[index])
    
    final_label = train_labels[index]
    
    if label_A != label_B:
    
        multi_hot_label_list.append(multi_hot_encoding.multi_hot_encode([[label_A,label_B]]))
        final_label_list.append(final_label)
    

In [11]:
reconstructed_multi_hot_decoder_model = keras.models.load_model("multi_hot_decoder_DNN")
reconstructed_multi_hot_decoder_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 512)               5632      
_________________________________________________________________
dense_5 (Dense)              (None, 9)                 4617      
Total params: 10,249
Trainable params: 10,249
Non-trainable params: 0
_________________________________________________________________


In [12]:
np_multi_hot_label_list = np.array(multi_hot_label_list)

In [14]:
np_multi_hot_label_list = np_multi_hot_label_list.reshape(longest_axis, 10)

In [16]:
bootstrap_predictions = reconstructed_multi_hot_decoder_model.predict(np_multi_hot_label_list)

In [19]:
true_count = 0
for index in range(len(bootstrap_predictions)):
    
    prediction = np.argmax(bootstrap_predictions[index])
    true_value = np.argmax(final_label_list[index])
    
    if prediction == true_value:
        true_count = true_count + 1

0
0
1
1
3
3
5
5
4
4
1
1
8
8
3
3
1
1
2
2
1
1
0
0
4
4
2
2
6
6
0
0
0
0
4
4
2
2
2
2
5
5
3
3
1
1
0
0
3
3
0
0
3
3
1
1
5
5
0
0
3
3
0
0
3
3
2
2
5
5
4
4
6
6
5
5
2
2
2
2
4
4
2
2
0
0
3
3
1
1
3
3
5
5
0
0
2
2
1
1
1
1
0
0
2
2
5
5
4
4
5
5
2
2
1
1
3
3
0
0
0
0
7
7
1
1
2
2
3
3
0
0
3
3
0
0
1
1
2
2
1
1
8
8
0
0
5
5
2
2
0
0
1
1
2
2
3
3
0
0
1
1
2
2
2
2
2
2
5
5
6
6
0
0
1
1
0
0
1
1
0
0
0
0
3
3
2
2
0
0
3
3
5
5
1
1
0
0
2
2
0
0
3
3
3
3
3
3
4
4
1
1
3
3
2
2
1
1
3
3
3
3
5
5
2
2
1
1
5
5
1
1
3
3
6
6
5
5
5
5
0
0
5
5
3
3
3
3
5
5
5
5
1
1
4
4
2
2
0
0
3
3
3
3
5
5
4
4
0
0
4
4
0
0
4
4
1
1
2
2
0
0
2
2
2
2
2
2
1
1
4
4
1
1
2
2
1
1
3
3
2
2
0
0
1
1
2
2
3
3
5
5
2
2
2
2
0
0
3
3
2
2
3
3
7
7
2
2
3
3
0
0
0
0
4
4
3
3
8
8
3
3
7
7
6
6
0
0
0
0
2
2
6
6
6
6
1
1
5
5
7
7
0
0
4
4
2
2
5
5
0
0
3
3
0
0
0
0
1
1
2
2
0
0
6
6
6
6
2
2
2
2
3
3
2
2
0
0
3
3
3
3
1
1
1
1
5
5
2
2
0
0
4
4
6
6
1
1
2
2
1
1
1
1
3
3
3
3
1
1
3
3
7
7
3
3
0
0
5
5
0
0
1
1
0
0
2
2
2
2
0
0
6
6
6
6
4
4
5
5
4
4
6
6
0
0
3
3
5
5
2
2
1
1
2
2
0
0
3
3
2
2
2
2
7
7
1
1
0
0
0
0
4
4
6
6
3
3
1
1


In [18]:
print("Accuracy = " + str(true_count/ len(bootstrap_predictions)))

Accuracy = 0.9996388688260459
